<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen + LangChain + PlayHT Use Case - Super AI Agent that Speaks

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks.

**`LangChain`** is an open-source framework designed for software developers engaged in AI and ML. It enables them to seamlessly integrate LLM with external components, facilitating the creation of LLM-driven applications.

**`PlayHT`** is a company serving the generative text to speech service.

Integrating them together, we are able to build a super cool AI agent that,

1. is knowledgeable in certain area
2. can **SPEAK**

This is the enhanced version of the AI Agent introduced in previous tutorial. We will build the audio feature on top of it. To learn more about it before starting this tutorial, please visit the following link:

[AutoGen + LangChain Use Case - Uniswap Protocol AI Agent](https://github.com/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb)



## Use Case - Uniswap Protocol AI Agent that Speaks

`Uniswap` is a decentralized exchange that allows users to trade Ethereum-based tokens.

In previous tutorial, we already built an AI Agent that can execute tasks require Uniswap protocol knowledge.

In this tutorial, let's make the agents answer in **audio**.

### Environment Preparation

In [ ]:
%pip install pyautogen~=0.1.0 docker langchain openai tiktoken chromadb pypdf simpleaudio numpy -q

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### Steps

#### 1. Build up a vector store with Uniswap V3 whitepaper.

In [ ]:
docs = PyPDFLoader('./uniswap_v3.pdf').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [ ]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs)

#### 2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [29]:
result = qa(({"question": "What is uniswap?"}))

In [30]:
result['answer']

' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine.'

#### 3. Define a function `answer_uniswap_question`

It takes a parameter `question`, calls the QA chain, and answer it by returning the answer from the chain response.

In [ ]:
def answer_uniswap_question(question):
  response = qa({"question": question})
  return response["answer"]

#### 4. Define a function convert_text_to_audio

In [ ]:
from typing import Generator, Iterable

import time
import threading
import os
import re
import numpy as np
import simpleaudio as sa

from pyht.client import Client, TTSOptions
from pyht.protos import api_pb2

def play_audio(data: Generator[bytes, None, None] | Iterable[bytes]):
    buff_size = 10485760
    ptr = 0
    start_time = time.time()
    buffer = np.empty(buff_size, np.float16)
    audio = None
    for i, chunk in enumerate(data):
        if i == 0:
            start_time = time.time()
            continue  # Drop the first response, we don't want a header.
        elif i == 1:
            print("First audio byte received in:", time.time() - start_time)
        for sample in np.frombuffer(chunk, np.float16):
            buffer[ptr] = sample
            ptr += 1
        if i == 5:
            # Give a 4 sample worth of breathing room before starting
            # playback
            audio = sa.play_buffer(buffer, 1, 2, 24000)
    approx_run_time = ptr / 24_000
    time.sleep(max(approx_run_time - time.time() + start_time, 0))
    if audio is not None:
        audio.stop()


def convert_text_to_audio(
    text: str
):
    text_partitions = re.split(r'[,.]', text)

    # Setup the client
    client = Client(os.environ['PLAY_HT_USER_ID'], os.environ['PLAY_HT_API_KEY'])

    # Set the speech options
    voice = "s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json"
    options = TTSOptions(voice=voice, format=api_pb2.FORMAT_WAV, quality="faster")

    # Get the streams
    in_stream, out_stream = client.get_stream_pair(options)

    # Start a player thread.
    audio_thread = threading.Thread(None, play_audio, args=(out_stream,))
    audio_thread.start()

    # Send some text, play some audio.
    for t in text_partitions:
        in_stream(t)
    in_stream.done()

    # cleanup
    audio_thread.join()
    out_stream.close()

    # Cleanup.
    client.close()
    return 0

In [ ]:
convert_text_to_audio("Welcome to the Uniswap V3 whitepaper.")

#### 5. Set up AutoGen agents with text-to-audio conversion function

In [ ]:
llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        },
        {
            "name": "convert_text_to_audio",
            "description": "Convert text to audio and speak it out loud",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to be converted and spoken out loud",
                    }
                },
                "required": ["text"],
            },
        }
    ],
}

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={
        "answer_uniswap_question": answer_uniswap_question,
        "convert_text_to_audio": convert_text_to_audio
    }
)

### It's time to let the agents SPEAK.

Now, let's user the user agent to ask the agents to write an introduction blog for `Uniswap` protocol v3, and **speak it out loudly**.

In [32]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. 
Find the answers to the 2 questions below, write an introduction based on them and speak it out loudly.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?

Start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. 
Find the answers to the 2 questions below, write an introduction based on them and speak it out loudly.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_uniswap_question *****
Arguments: 
{
  "question": "What is Uniswap?"
}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_uniswap_question...
user_proxy (to assistant):

***** Response from calling function "answer_uniswap_question" *****
 Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine.
********************************************************************

-----------------------